In [12]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_leaflet as dl
from animal_shelter import AnimalShelter

# Initialize database connection
username = "aacuser"
password = "adam1"
shelter = AnimalShelter(username, password)

# Fetch initial data
initial_data = pd.DataFrame.from_records(shelter.read({}))

# App Initialization
app = JupyterDash(__name__)

# App Layout
app.layout = html.Div([
    html.Center(html.B(html.H1('Dashboard'))),
    html.Hr(),

    # Filtering Options
    html.Div([
        html.Label('Filter by Rescue Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset', 'value': 'Reset'},
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster Rescue', 'value': 'Disaster'}
            ],
            value='Reset',
            inline=True
        )
    ]),

    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in initial_data.columns],
        data=initial_data.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_data={'whiteSpace': 'normal', 'height': 'auto'},
        style_header={'fontWeight': 'bold'},
        row_selectable='single',
        page_action='native',
        page_current=0
    ),

    html.Hr(),

    # Charts and Map
    html.Div([
        html.Div(id='chart-id', style={'width': '50%', 'display': 'inline-block'}),
        html.Div(id='map-id', style={'width': '50%', 'display': 'inline-block'})
    ])
])

# Callbacks for filtering
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_table(filter_type):
    if filter_type == 'Reset':
        data = shelter.read({})
    elif filter_type == 'Water':
        data = shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
            "outcome_type": {"$nin": ["Euthanasia"]}
        })
    elif filter_type == 'Mountain':
        data = shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
            "outcome_type": {"$nin": ["Euthanasia"]}
        })
    elif filter_type == 'Disaster':
        data = shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
            "outcome_type": {"$nin": ["Euthanasia"]}
        })
    else:
        data = shelter.read({})

    return pd.DataFrame.from_records(data).to_dict('records')

# Callback for chart
@app.callback(
    Output('chart-id', 'children'),
    [Input('datatable-id', 'data')]
)
def update_chart(data):
    dff = pd.DataFrame(data)
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return dcc.Graph(figure=fig)

# Callback for map
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'data'), Input('datatable-id', 'selected_rows')]
)
def update_map(data, selected_rows):
    dff = pd.DataFrame(data)
    if not selected_rows:
        selected_rows = [0]
    row = selected_rows[0]
    return dl.Map(center=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], zoom=10, children=[
        dl.TileLayer(),
        dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
            dl.Tooltip(dff.iloc[row]['animal_type']),
            dl.Popup([html.H1("Animal Details"), html.P(dff.iloc[row]['breed'])])
        ])
    ])

# Run App
app.run_server(port=8050)



Dash app running on http://127.0.0.1:8050/
